In [41]:
import sys 
sys.path.append('..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import torch as ch
from torch import Tensor
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.utils.data import DataLoader
from delphi.stats.truncated_regression import truncated_regression
from delphi.utils.datasets import TruncatedRegressionDataset
from delphi.oracle import Left, Interval

In [54]:
STORE_PATH = '/home/pstefanou/knownVsUnknown'
STORE_TABLE_NAME = 'rebuttal4'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'known_param_mse': float,
    'unknown_param_mse': float, 
    'ols_param_mse': float,
    'alpha': float, 
    'var': float, 
})

Logging in: /home/pstefanou/knownVsUnknown/0dc8850b-f527-4ac0-b73e-5964856a805f


In [ ]:
# regression parameters
num_samples, dims = 1000, 10
# noise_var = Tensor([9.0])
W = ch.ones(dims, 1)
W0 = ch.ones(1, 1)
phi = Left(Tensor([1.0]))


# perform each experiment a total of 10 times
for iter_ in range(10):        
    for var in range(1, 10):
        # generate data
        X = MultivariateNormal(ch.zeros(dims), ch.eye(dims)/dims).sample(ch.Size([num_samples]))
        y = X.mm(W) + W0 + Normal(ch.zeros(1), ch.sqrt(Tensor([var]))).sample(ch.Size([num_samples]))
        # truncate
        # phi = Interval(Tensor([1.0]), Tensor([2.0]))
        indices = phi(y).nonzero(as_tuple=False).flatten()
        y_trunc, x_trunc = y[indices], X[indices]


        # truncated linear regression with known noise variance
        reg = LinearRegression() 
        reg.fit(x_trunc, y_trunc)
        emp_var = ch.var(ch.from_numpy(reg.predict(x_trunc)) - y_trunc, dim=0).unsqueeze(0)

        # experiment parameters
        args = Parameters({ 
            'alpha': Tensor([y_trunc.size(0)/num_samples]), 
            'phi': phi, 
            'epochs': 75,
            'num_workers': 2, 
            'batch_size': 100,
            'bias': True,
            'num_samples': 100,
            'clamp': True, 
            'radius': 5.0, 
            'lr': 1e-1,
            'unknown': False, 
            'var': emp_var,
            'shuffle': True
        })


        # dataset 
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=args.unknown)
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)

        trunc_reg = truncated_regression(phi=phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=num_samples, var=args.var)
        results = trunc_reg.fit(S)
        w_ = results.weight.detach()
        w0_ = results.bias.detach()

        # truncated linear regression with unknown noise variance
        # experiment parameters
        args = Parameters({ 
            'alpha': Tensor([y_trunc.size(0)/num_samples]), 
            'phi': phi, 
            'epochs': 75,
            'num_workers': 2, 
            'batch_size': 100,
            'bias': True,
            'num_samples': 100,
            'clamp': True, 
            'radius': 5.0, 
            'var_lr': 1e-2,
            'lr': 1e-1,
            'unknown': True, 
            'shuffle': True
        })

        # dataset 
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=args.unknown)
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)

        trunc_reg = truncated_regression(phi=phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=num_samples, var=args.var)

        results = trunc_reg.fit(S)
        var_ = results.lambda_.inverse().detach().cpu()
        w = results.v.detach().cpu()*var_
        w0 = results.bias.detach().cpu()*var_
        
        # calculate metrics 
        real_params = ch.cat([W, W0])
        ols_params = ch.cat([data.w, data.w0.unsqueeze(0)])
        unknown_params = ch.cat([w, w0])
        known_params = ch.cat([w_.t(), w0_.unsqueeze(0)])
        unknown_param_mse = mean_squared_error(unknown_params, real_params)
        ols_param_mse = mean_squared_error(ols_params, real_params)
        known_param_mse = mean_squared_error(known_params, real_params)

        store[STORE_TABLE_NAME].append_row({ 
            'known_param_mse': known_param_mse,
            'unknown_param_mse': unknown_param_mse,
            'ols_param_mse': ols_param_mse,
            'alpha': float(args.alpha.flatten()),
            'var': float(var), 
        })

# close current store
store.close()

Epoch:1 | Loss 0.2531 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 16.25it/s]
Epoch:2 | Loss 0.2531 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.30it/s]
Epoch:3 | Loss 0.2534 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.22it/s]
Epoch:4 | Loss 0.2535 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.28it/s]
Epoch:5 | Loss 0.2552 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.21it/s]
Epoch:6 | Loss 0.2594 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.83it/s]
Epoch:7 | Loss 0.2639 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.81it/s]
Epoch:8 | Loss 0.2690 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.39it/s]
Epoch:9 | Loss 0.2766 | Train1 0.294 | Train5 -1

Epoch:63 | Loss 0.3931 | Train1 0.290 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.44it/s]
Epoch:64 | Loss 0.3945 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.32it/s]
Epoch:65 | Loss 0.3918 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.22it/s]
Epoch:66 | Loss 0.3926 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.91it/s]
Epoch:67 | Loss 0.3920 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 16.58it/s]
Epoch:68 | Loss 0.3919 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.18it/s]
Epoch:69 | Loss 0.3951 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.26it/s]
Epoch:70 | Loss 0.3922 | Train1 0.288 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.09it/s]
Epoch:71 | Loss 0.3943 | Train1 0.288 | 

Epoch:56 | Loss 0.2729 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 15.64it/s]
Epoch:57 | Loss 0.2753 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.84it/s]
Epoch:58 | Loss 0.2772 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 15.85it/s]
Epoch:59 | Loss 0.2767 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.71it/s]
Epoch:60 | Loss 0.2770 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.84it/s]
Epoch:61 | Loss 0.2747 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.56it/s]
Epoch:62 | Loss 0.2747 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.42it/s]
Epoch:63 | Loss 0.2741 | Train1 0.294 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.66it/s]
Epoch:64 | Loss 0.2707 | Train1 0.294 | 

Epoch:49 | Loss 0.5109 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  8.61it/s]
Epoch:50 | Loss 0.5115 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 10.58it/s]
Epoch:51 | Loss 0.5107 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 10.10it/s]
Epoch:52 | Loss 0.5122 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  7.24it/s]
Epoch:53 | Loss 0.5118 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.13it/s]
Epoch:54 | Loss 0.5130 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  8.43it/s]
Epoch:55 | Loss 0.5122 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  8.63it/s]
Epoch:56 | Loss 0.5115 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  8.38it/s]
Epoch:57 | Loss 0.5121 | Train1 0.242 | 

Epoch:42 | Loss 0.5007 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.45it/s]
Epoch:43 | Loss 0.4999 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.67it/s]
Epoch:44 | Loss 0.4968 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.23it/s]
Epoch:45 | Loss 0.4975 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.65it/s]
Epoch:46 | Loss 0.5043 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.82it/s]
Epoch:47 | Loss 0.5000 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.66it/s]
Epoch:48 | Loss 0.5118 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 12.30it/s]
Epoch:49 | Loss 0.5149 | Train1 0.242 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00,  7.12it/s]
Epoch:50 | Loss 0.5190 | Train1 0.242 | 

...

Epoch:15 | Loss 0.3257 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 16.81it/s]
Epoch:16 | Loss 0.3322 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 16.71it/s]
Epoch:17 | Loss 0.3423 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.03it/s]
Epoch:18 | Loss 0.3506 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.54it/s]
Epoch:19 | Loss 0.3512 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 16.62it/s]
Epoch:20 | Loss 0.3582 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.46it/s]
Epoch:21 | Loss 0.3579 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.54it/s]
Epoch:22 | Loss 0.3645 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 17.34it/s]
Epoch:23 | Loss 0.3722 | Train1 0.281 | 

Epoch:8 | Loss 0.2671 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 49.24it/s]
Epoch:9 | Loss 0.2673 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 46.48it/s]
Epoch:10 | Loss 0.2686 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 47.15it/s]
Epoch:11 | Loss 0.2685 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 46.58it/s]
Epoch:12 | Loss 0.2684 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 46.57it/s]
Epoch:13 | Loss 0.2692 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 46.07it/s]
Epoch:14 | Loss 0.2700 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 48.06it/s]
Epoch:15 | Loss 0.2702 | Train1 0.281 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 5/5 [00:00<00:00, 46.73it/s]
Epoch:16 | Loss 0.2725 | Train1 0.281 | Tr

Epoch:1 | Loss 0.5174 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.42it/s]
Epoch:2 | Loss 0.5174 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.88it/s]
Epoch:3 | Loss 0.5175 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.66it/s]
Epoch:4 | Loss 0.5176 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.47it/s]
Epoch:5 | Loss 0.5188 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.04it/s]
Epoch:6 | Loss 0.5195 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.31it/s]
Epoch:7 | Loss 0.5219 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 13.01it/s]
Epoch:8 | Loss 0.5252 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.24it/s]
Epoch:9 | Loss 0.5290 | Train1 0.263 | Train5 -1

Epoch:69 | Loss 0.5999 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.26it/s]
Epoch:70 | Loss 0.6004 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.70it/s]
Epoch:71 | Loss 0.6011 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 18.00it/s]
Epoch:72 | Loss 0.6010 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.72it/s]
Epoch:73 | Loss 0.6031 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.44it/s]
Epoch:74 | Loss 0.6027 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.70it/s]
Epoch:75 | Loss 0.6028 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.30it/s]
Epoch:1 | Loss 0.5174 | Train1 0.263 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 54.54it/s]
Epoch:2 | Loss 0.5174 | Train1 0.263 | Tr

Epoch:62 | Loss 0.7572 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.56it/s]
Epoch:63 | Loss 0.7686 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 17.35it/s]
Epoch:64 | Loss 0.7562 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.66it/s]
Epoch:65 | Loss 0.7429 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.72it/s]
Epoch:66 | Loss 0.7324 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.37it/s]
Epoch:67 | Loss 0.7284 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.88it/s]
Epoch:68 | Loss 0.7259 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.30it/s]
Epoch:69 | Loss 0.7313 | Train1 0.255 | Train5 -1.000 | Reg term: 0.0 ||: 100%|██████████| 6/6 [00:00<00:00, 14.31it/s]
Epoch:70 | Loss 0.7371 | Train1 0.255 | 

In [ ]:
# store.close()
reader = CollectionReader(STORE_PATH)
results = reader.df(STORE_TABLE_NAME)
results.head()

In [ ]:
# plot results for regression parameter MSE
sns.lineplot(data=unknown, x='var', y='unknown_param_mse', label='unknown', color='blue')
sns.lineplot(data=known, x='var', y='known_param_mse', label='known', color='purple')
ax = sns.lineplot(data=ols, x='var', y='ols_param_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error')
plt.show()

In [27]:
known, unknown, ols = results[['known_param_mse', 'var']], results[['unknown_param_mse', 'var']], results[['ols_param_mse', 'var']]

In [49]:
store.close()

In [39]:
for v in range(1, 10): 
    unknown = unknown.drop(unknown[unknown['var'] == v].idxmax())
    unknown = unknown.drop(unknown[unknown['var'] == v].idxmin())
    known = known.drop(known[known['var'] == v].idxmax())
    known = known.drop(known[known['var'] == v].idxmin())
    ols = ols.drop(ols[ols['var'] == v].idxmax())
    ols = ols.drop(ols[ols['var'] == v].idxmin())

In [50]:
store

In [52]:
s = Store('/home/pstefanou/knownVsUnknown/', 'c1d215fa-6aa2-4472-ae90-ecbad434b533')

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1400, in H5F__open
    unable to open file
  File "H5Fint.c", line 1615, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 941, in H5FD_sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file '/home/pstefanou/knownVsUnknown/c1d215fa-6aa2-4472-ae90-ecbad434b533/store.h5'